<img src="images/splash3.png">
Requests data for the past hour and inserts this into SQL. To be ran hourly.


<p><b>What you were doing:</b></p>
<p>&nbsp; &nbsp; only write new tweets, in bottom cell </p>
<p>&nbsp; &nbsp; possible issue: only fetches tweets with 140+ character length? </p>

## Setup

In [115]:
import sqlite3 as lite
import sys
import requests
import pandas as pd
import numpy as np
import datetime
import json
import tweepy

In [116]:
# Twitter stuff
CONSUMER_KEY = 'hiZmnSOV63GdqWPqW5MN533T1'# TWITTER_CONSUMER_TOKEN
CONSUMER_SECRET = 'GJa7snhSw8jDQxTVhvoHJyIyJZBjLQikAeuNL0Y3o0Aw61zuOr' # TWITTER_CONSUMER_SECRET
ACCESS_TOKEN = '933287551212904448-uWflGB32dNF8rbgmBxJk1ql5AqemO14' # TWITTER_ACCESS_TOKEN
ACCESS_SECRET = 'boUb7syKFxrCmaJN3T3SHsWu6rDQQfdPxertNIEnXR6X9' # TWITTER_ACCESS_SECRET

## Class & function definitions


In [118]:
class Coin:
    def __init__(self, symbol, comparison_symbol, limit, aggregate):
        self.symbol = symbol
        self.comparison_symbol = comparison_symbol
        self.limit = limit
        self.aggregate = aggregate
        
    def fetch_marketdata(self):    
        url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
                .format(self.symbol.upper(), self.comparison_symbol.upper(), self.limit, self.aggregate)       
        page = requests.get(url)
        data = page.json()['Data']
        df = pd.DataFrame(data)
        df['mid'] = df[["high", "low"]].mean(axis=1)
        df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
        return df
    
    def fetch_tweets(self):
        # Create extractor object
        extractor = twitter_setup()
        # Create a tweet list
        tweets = extractor.search(q ="${}".format(coin_instances[coin].symbol), count=20, tweet_mode="extended", lang="en")
        
        def fulltext(tweet):
            try:
                text = tweet.retweeted_status.full_text
                return(text)
            except:
                text = tweet.full_text
                return(text)

        df = pd.DataFrame(data=[fulltext(tweet) for tweet in tweets], columns=['Tweets'])
        
        df['Date'] = np.array([tweet.created_at for tweet in tweets])
        df['Date'] = df['Date'].astype(np.int64) // 10**9 # Convert to unix
        df['ID']   = np.array([tweet.id for tweet in tweets])
        df['Len']  = np.array([len(tweet.full_text) for tweet in tweets])
        df['Reply_To_Status'] = np.array([tweet.in_reply_to_user_id for tweet in tweets])
        df['Reply_To_Name'] = np.array([tweet.in_reply_to_screen_name for tweet in tweets])
        df['Coordinates'] = np.array([tweet.coordinates for tweet in tweets])
        df['Retweet_Count'] = np.array([tweet.retweet_count for tweet in tweets])
        df['User_Verified'] = np.array([tweet.user.verified for tweet in tweets])
        df['User_Location'] = np.array([tweet.user.location for tweet in tweets])
        df['User_Followers_Count'] = np.array([tweet.user.followers_count for tweet in tweets])
        df['User_Favorite_Count'] = np.array([tweet.user.favourites_count for tweet in tweets])
        df['Source'] = np.array([tweet.source for tweet in tweets])
        df['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
        df['Favorite_Count'] = np.array([tweet.favorite_count for tweet in tweets])
        df['RTs'] = np.array([tweet.retweet_count for tweet in tweets])        
        return df
        
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication
    api = tweepy.API(auth)
    return api    
    
    
def market_to_db(market_data):
    con = None
    con = lite.connect('/hdd/db/crypto.db') #Connect to DB

    
    with con: #Dump to SQL
        cur = con.cursor()
        cur.execute("SELECT time FROM {} ORDER BY time DESC LIMIT 1".format(coin_instances[coin].symbol))
        rows = cur.fetchall()
        latest_db_entry = list(rows[0])
        latest_db_entry = int(latest_db_entry[0])
        if latest_db_entry < market_data.iloc[0]['time']:
            #print(latest_db_entry, "is lower than", market_data.iloc[0]['time'], "<- Time to write!") # Debug
            market_data.to_sql(name=coin_instances[coin].symbol, con=con, if_exists='append', index=True)
            #print("wrote to DB for coin", coin_instances[coin].symbol) # Debug 
        else:
            pass
            #print(latest_db_entry, "(latest db entry) is the same or higher than", market_data.iloc[0]['time'], "(fetched data), skipping write for", coinlist[coin])
            
def tweets_to_db(df_tweets):
    con = None
    con = lite.connect('/hdd/db/crypto.db') #Connect to DB
    
    with con: #Dump to SQL
        cur = con.cursor()
        print("Going to write tweets to {}_tweets".format(coin_instances[coin].symbol))
        df_tweets.to_sql(name="{}_tweets".format(coin_instances[coin].symbol), con=con, if_exists='append', index=True)

## Fetch data & store in db

In [119]:
fh = open("/hdd/db/tracked_coins.json", 'r')
coinlist = json.load(fh)
coin_instances = [Coin(coinlist[coin], 'BTC','1','1') for coin in range(len(coinlist))] # Make objects. Take all coins in coinlist, convert to BTC, poll 1 entry and aggregate per 1-hour period.

#Get data
#for coin in range(len(coin_instances)):
for coin in range(5):
    try:
        market_data = coin_instances[coin].fetch_marketdata() # Get coindata
        df_tweets = coin_instances[coin].fetch_tweets() # Get tweets
        market_data = market_data.drop(market_data.index[len(market_data)-1]) # Only keep the latest timestamped column
        market_to_db(market_data)
        tweets_to_db(df_tweets)
    except Exception as e:
        print("Skipping coin", coin_instances[coin], ", Error: ")
        print(e)        

Going to write tweets to KCS_tweets
Going to write tweets to ETH_tweets
Going to write tweets to SC_tweets
Going to write tweets to DGB_tweets
Going to write tweets to REP_tweets


In [70]:

df_tweets = coin_instances[80].fetch_tweets() # Get tweets

In [64]:
df_tweets['Date'] = df_tweets['Date'].astype(np.int64) // 10**9

In [71]:
df_tweets.tail()

,Tweets,Date,ID,Len,Reply_To_Status,Reply_To_Name,Coordinates,Retweet_Count,User_Verified,User_Location,User_Followers_Count,User_Favorite_Count,Source,Likes,Favorite_Count,RTs
15,1BTC --&gt; 30BTC JUST 1 MONTH\n\nJoin : https...,1529788397,1010631904670838785,235,None,None,None,0,False,,85,1,twittbot.net,0,0,0
16,Long/Short Bitcoin movements with up to 100x L...,1529787728,1010629100749316096,247,None,None,None,0,False,,3,0,Twitter Web Client,0,0,0
17,Buy this coin in Binance right now!\nPossible ...,1529787619,1010628642915979264,259,None,None,None,0,False,,60546,654,BinanceBEST024,0,0,0
18,Register with Binance and you can trade for as...,1529787609,1010628600536666114,265,None,None,None,0,False,,116,0,IFTTT,0,0,0
19,Litecoin $LTC is valued at: $81.93 \n\nWe veri...,1529787517,1010628215491186693,267,None,None,None,0,False,,82,0,lenovovovo38,0,0,0


In [66]:
df_tweets['Tweets'].iloc[4]

'Litecoin $LTC is valued at: $83.17 \n\nWe checked! Binance registration is currently open 🙌 💰  \n\n➡️ https://t.co/eL2lTAAK5E\n\n$COB $UTK $ANT $PPC $XUC $SKY $LSK $ART $WTC $REP $XMR $SWT $ADT $MOD $EDG $ION $EGC $EFL $QRK $QASH $DCR $ABY $DYN https://t.co/Aar0wxCPZ6'

In [73]:
def market_to_dbs(market_data):
    con = None
    con = lite.connect('/hdd/db/crypto.db') #Connect to DB

    
    with con: #Dump to SQL
        cur = con.cursor()
        cur.execute("SELECT time FROM {} ORDER BY time DESC LIMIT 1".format(coin_instances[coin].symbol))
        rows = cur.fetchall()
        latest_db_entry = list(rows[0])
        latest_db_entry = int(latest_db_entry[0])
        if latest_db_entry < market_data.iloc[0]['time']:
            #print(latest_db_entry, "is lower than", market_data.iloc[0]['time'], "<- Time to write!") # Debug
            market_data.to_sql(name=coin_instances[coin].symbol, con=con, if_exists='append', index=True)
            #print("wrote to DB for coin", coin_instances[coin].symbol) # Debug 
        else:
            pass
            #print(latest_db_entry, "(latest db entry) is the same or higher than", market_data.iloc[0]['time'], "(fetched data), skipping write for", coinlist[coin])
            

con = None
con = lite.connect('/hdd/db/crypto.db') #Connect to DB

with con: #Dump to SQL
    cur = con.cursor()
    
    cur.execute("SELECT Date FROM {}_tweets ORDER BY Date DESC LIMIT 1".format(coin_instances[coin].symbol))
    rows = cur.fetchall()
    latest_db_entry = list(rows[0])
    latest_db_entry = int(latest_db_entry[0])
    print(latest_db_entry)
    if latest_db_entry < df_tweets.iloc[0]['Date']:
        print(latest_db_entry, "is lower than", df_tweets.iloc[0]['Date'], "<- Time to write!") # Debug
        df_tweets.to_sql(name="{}_tweets".format(coin_instances[coin].symbol), con=con, if_exists='append', index=True)   # Change to only new tweets out of DF
        print("wrote to DB_tweets for coin", coin_instances[coin].symbol) # Debug
    else:
        print(latest_db_entry, "(latest db entry) is the same or higher than", market_data.iloc[0]['time'], "(fetched data), skipping write for", coinlist[coin], "AND df_tweets.iloc[0]['Date']", df_tweets.iloc[0]['Date']) 
        pass

1529794827
1529794827 (latest db entry) is the same or higher than 1529787600 (fetched data), skipping write for REP AND df_tweets.iloc[0]['Date'] 1529794827


In [112]:
con = lite.connect('/hdd/db/crypto.db') #Connect to DB

date = pd.read_sql_query("SELECT Date FROM {}_tweets ORDER BY Date DESC LIMIT 1".format(coin_instances[coin].symbol), con)
latest_db_entry = pd.to_datetime(date['Date'].values[0], unit = 's')

df_tweets['Date'] = pd.to_datetime(df_tweets['Date'], unit = 's')
df_tweets.loc[df_tweets['Date'] >= latest_db_entry]

,Tweets,Date,ID,Len,Reply_To_Status,Reply_To_Name,Coordinates,Retweet_Count,User_Verified,User_Location,User_Followers_Count,User_Favorite_Count,Source,Likes,Favorite_Count,RTs
0,Buy this coin in Binance right now!\nPossible ...,2018-06-23 23:00:27,1010658875513671680,259,None,None,None,0,False,,60546,654,BinanceBEST024,0,0,0
1,Register with Binance and you can trade for as...,2018-06-23 23:00:20,1010658845113356288,265,None,None,None,0,False,,116,0,IFTTT,0,0,0
2,Bitcoin Cash $BCH price: $764.58 \n\nQUICK! Si...,2018-06-23 22:57:17,1010658076045832192,265,None,None,None,0,False,,37,0,modcfjodfjghfoijhioj,0,0,0
3,Current $WAVES price: $2.88 \n\nBinance is the...,2018-06-23 22:34:35,1010652364171300864,273,None,None,None,0,False,,15,0,wert3445983444982189,0,0,0
4,You still haven't registered at the #1 Crypto ...,2018-06-23 22:10:58,1010646423648468992,270,None,None,None,0,False,,3,0,Twitter Web Client,0,0,0
5,Replicate successful traders automatically wit...,2018-06-23 22:06:25,1010645278746013698,276,None,None,None,0,False,,1,0,Twitter Web Client,0,0,0
6,Lisk $LSK price: $5.58 \n\nBinance registratio...,2018-06-23 22:01:20,1010643997294039041,253,None,None,None,0,False,,211,5,twitterbotmaster21,0,0,0
7,Litecoin $LTC is valued at: $83.17 \n\nWe chec...,2018-06-23 22:01:00,1010643912304807936,262,None,None,None,0,False,,24,0,95fg9h5f9h29fgh9fg5h95,0,0,0
8,Register with Binance and you can trade for as...,2018-06-23 22:00:24,1010643764573081601,265,None,None,None,0,False,,116,0,IFTTT,0,0,0
9,Buy this coin in Binance right now!\nPossible ...,2018-06-23 22:00:23,1010643759854444544,259,None,None,None,0,False,,60546,654,BinanceBEST024,0,0,0


In [111]:
latest_db_entry

Timestamp('2018-06-23 23:00:27')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package redis


In [114]:
# wat je moet doen: functie schrijven die dataframe sliced op timestamps die nieuwer zijn dan de latest_db_entry timestamp
# vind het punt en snij daar de helft af
# functie aanroepen in plaats van df_tweets.to_sql(name="{}_tweets.format)


df_tweets['Date'] = pd.to_datetime(df_tweets['Date'], unit = 's')

df_tweets

    

,Tweets,Date,ID,Len,Reply_To_Status,Reply_To_Name,Coordinates,Retweet_Count,User_Verified,User_Location,User_Followers_Count,User_Favorite_Count,Source,Likes,Favorite_Count,RTs
0,Buy this coin in Binance right now!\nPossible ...,2018-06-23 23:00:27,1010658875513671680,259,None,None,None,0,False,,60546,654,BinanceBEST024,0,0,0
1,Register with Binance and you can trade for as...,2018-06-23 23:00:20,1010658845113356288,265,None,None,None,0,False,,116,0,IFTTT,0,0,0
2,Bitcoin Cash $BCH price: $764.58 \n\nQUICK! Si...,2018-06-23 22:57:17,1010658076045832192,265,None,None,None,0,False,,37,0,modcfjodfjghfoijhioj,0,0,0
3,Current $WAVES price: $2.88 \n\nBinance is the...,2018-06-23 22:34:35,1010652364171300864,273,None,None,None,0,False,,15,0,wert3445983444982189,0,0,0
4,You still haven't registered at the #1 Crypto ...,2018-06-23 22:10:58,1010646423648468992,270,None,None,None,0,False,,3,0,Twitter Web Client,0,0,0
5,Replicate successful traders automatically wit...,2018-06-23 22:06:25,1010645278746013698,276,None,None,None,0,False,,1,0,Twitter Web Client,0,0,0
6,Lisk $LSK price: $5.58 \n\nBinance registratio...,2018-06-23 22:01:20,1010643997294039041,253,None,None,None,0,False,,211,5,twitterbotmaster21,0,0,0
7,Litecoin $LTC is valued at: $83.17 \n\nWe chec...,2018-06-23 22:01:00,1010643912304807936,262,None,None,None,0,False,,24,0,95fg9h5f9h29fgh9fg5h95,0,0,0
8,Register with Binance and you can trade for as...,2018-06-23 22:00:24,1010643764573081601,265,None,None,None,0,False,,116,0,IFTTT,0,0,0
9,Buy this coin in Binance right now!\nPossible ...,2018-06-23 22:00:23,1010643759854444544,259,None,None,None,0,False,,60546,654,BinanceBEST024,0,0,0


In [82]:
for row in df_tweets['Date']:
    print (row)

1529794827
1529794820
1529794637
1529793275
1529791858
1529791585
1529791280
1529791260
1529791224
1529791223
1529790330
1529789803
1529789799
1529789709
1529788612
1529788397
1529787728
1529787619
1529787609
1529787517
